In [24]:
import pandas as pd
import statsmodels.api as sm
from scipy.stats import chi2_contingency

import seaborn as sns

Команда приложения разработала новый алгоритм для поиска наиболее подходящих анкет. Для проверки работы алгоритма был проведен АБ-тест. Все пользователи были разделены на две группы. Пользователи в группе с номером 0 пользовались приложением со старым алгоритмом. Все пользователи в группе 1 пользовались приложением с новым алгоритмом для поиска анкет.

Ваша задача – оценить, правда ли, что новый алгоритм улучшил качество сервиса. Для этого нужно выбрать метрики, которые отвечают за качество сервиса, и статистически сравнить эти метрики в двух группах.

В данных находится выгрузка логов взаимодействия пользователей друг с другом. Для каждой пары пользователей указано, из какой они группы АБ-теста и случился ли у них мэтч.

Результат вашей работы – аналитическое заключение с ответом на вопрос, стоит ли включать новую систему поиска анкет на всех пользователей.

In [2]:
dating_df = pd.read_csv('dating_data.csv')

In [3]:
dating_df.shape

(14514, 4)

In [4]:
dating_df.head()

,user_id_1,user_id_2,group,is_match
0,79,91,1,1
1,716,353,1,1
2,423,677,0,0
3,658,165,1,1
4,969,155,0,1


### Выбор метрики

Целевая метрика – мэтч. Дополнительные целевые метрики или прокси-метрики я выбрать не могу, так как по сути не из чего: ```user_id_1``` и ```user_id_2``` представляют из себя пару, и их нельзя разбивать. А оставшиеся – это ```group``` и ```is_match```.   

### Выбор теста

Определяю тип переменной. В данном случае оценивается факт – случился мэтч или нет, это "конверсия", а не метрическая переменная. Поэтому я могу применить z-тест или хи-квадрат. Я решила провести оба.

Также я не могу здесь проверять распределение, потому что ```is_match``` по сути бинарная переменная.

Для проведения z-теста нужно создать таблицу сопряжённости, я это делаю с помощью функции ```crosstab```. А потом провести сам z-тест. 

In [18]:
contingency_table = pd.crosstab(dating_df['group'], dating_df['is_match'])

In [19]:
contingency_table

is_match,0,1
group,,
0,3858,934
1,5813,3909


In [20]:
sm.stats.proportions_ztest(contingency_table[1], contingency_table.sum(axis=1))

(-24.89217439232899, 9.0433754456208e-137)

Первая величина – это z-статистика, вторая – стат.значимость. Стат.значимость близка к нулю, то есть можно сделать вывод, что между двумя группами есть значимые различия. Новый алгоритм действительно улучшил качество сервиса. Можно включить новую систему для всех пользователей. 

### Хи-квадрат

In [25]:
chi2_contingency(contingency_table)

(618.6889141576198,
 1.4418299163662586e-136,
 1,
 array([[3193.01584677, 1598.98415323],
        [6477.98415323, 3244.01584677]]))

Тут стат.значимость так же получилась близкой к нулю, вывод делаю тот же (между группами есть значимые различия, новый алгоритм улучшил качество сервиса и можно включить его для всех пользователей).